In [1]:
import dgl
from dgl.data import DGLDataset
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv,MaxPooling
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import time
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import os
import pandas
import networkx as nx
%matplotlib inline

In [4]:
g=dgl.DGLGraph()
g.add_nodes(3)
g.add_edge(0,1)
g.add_edge(1,0)
g.add_edge(1,2)
g.add_edge(2,1)
g.add_edge(0,2)
g.add_edge(2,0)

In [46]:
rand_feat=torch.randint(0,10,(3,3,9,9),dtype=torch.float32)
g.ndata['f']=rand_feat

In [33]:
g.ndata['f']

tensor([[[[3., 5., 4., 4., 3., 7., 8., 3., 2.],
          [2., 1., 0., 5., 0., 2., 1., 3., 1.],
          [5., 1., 6., 4., 9., 0., 1., 8., 8.],
          [6., 9., 5., 5., 5., 2., 2., 5., 8.],
          [8., 5., 2., 4., 4., 0., 7., 6., 0.],
          [2., 8., 9., 7., 2., 7., 2., 5., 4.],
          [3., 6., 3., 0., 3., 1., 6., 3., 2.],
          [1., 9., 7., 8., 2., 0., 4., 8., 7.],
          [1., 9., 0., 0., 7., 5., 1., 8., 6.]],

         [[6., 7., 3., 9., 1., 7., 7., 4., 6.],
          [5., 7., 4., 6., 7., 6., 8., 0., 4.],
          [8., 1., 8., 1., 3., 6., 8., 4., 2.],
          [6., 5., 5., 1., 2., 1., 1., 2., 7.],
          [3., 3., 6., 3., 4., 8., 5., 9., 9.],
          [4., 0., 9., 1., 4., 1., 8., 3., 4.],
          [7., 7., 2., 3., 0., 2., 8., 6., 9.],
          [5., 4., 6., 2., 7., 8., 6., 4., 4.],
          [6., 3., 6., 1., 8., 3., 8., 8., 0.]],

         [[9., 3., 5., 7., 1., 7., 4., 3., 7.],
          [6., 6., 0., 0., 8., 3., 9., 6., 5.],
          [8., 8., 4., 2., 5., 0., 2

In [49]:
conv=GraphConv(9,10)
pred=conv(g,g.ndata['f'])
g.ndata['h'] = pred

In [50]:
print(pred)
print(pred.shape)

tensor([[[[ -4.4602,   7.1812,  -1.2192,   0.2291,  -9.1119,  -9.3166,   4.5859,
             5.7091,   6.1033,   6.1876],
          [ -6.3254,   5.8962,  -3.8534,  -0.6243,  -5.4745,  -5.4885,   4.7048,
             2.4703,   3.2303,   2.6653],
          [ -2.3688,   5.1588,   0.0315,  -0.2406,  -8.4240, -11.1045,   4.1214,
            -0.0452,   4.4540,   4.7002],
          [ -5.3204,   5.8384,  -0.7834,   0.9014,  -7.5164,  -6.0196,   2.9560,
             4.4972,   4.4498,   9.2207],
          [ -0.2489,   0.9037,  -0.2649,  -2.7829,  -7.7364,  -2.2177,   0.8190,
             0.8070,   3.0704,   3.4042],
          [ -7.0873,   5.9265,  -1.4485,  -0.6669,  -7.7173,  -0.2722,   4.3142,
             2.4681,   3.7828,   8.6944],
          [ -1.5586,   6.9119,   2.3686,  -0.4413,  -8.6090,  -7.3203,   3.3788,
             3.5108,   4.3643,   6.4477],
          [ -5.6817,   4.9713,  -1.6772,   0.6323,  -8.2880,  -4.2338,   4.3959,
             3.5989,   6.3505,   5.4910],
          [ -1.3

In [51]:
pred=dgl.mean_nodes(g,'h')
print(pred)

tensor([[[[ -4.7399,   7.7535,  -1.0515,   0.3738,  -8.5452,  -7.5845,   2.9402,
             4.7455,   5.1380,   6.4582],
          [ -3.5028,   4.7383,  -1.3522,  -1.7864,  -6.3709,  -5.1336,   3.4002,
             2.4373,   2.6551,   2.5388],
          [ -1.8073,   4.8102,   0.6276,  -0.7678,  -9.3675, -11.2203,   5.3734,
             1.3229,   5.3704,   4.4160],
          [ -4.7905,   5.4722,  -0.0213,   1.3394,  -7.3264,  -5.6679,   3.5955,
             3.1285,   4.9936,   7.9722],
          [ -3.0860,   4.3977,  -1.0621,  -2.9593,  -8.2597,  -3.3402,   1.6475,
             2.4937,   2.4625,   4.7591],
          [ -6.5501,   5.0096,  -1.5809,  -0.5058,  -7.7866,  -1.7820,   4.0882,
             1.0907,   3.9930,   7.5125],
          [ -4.4788,   5.0837,  -0.4952,  -0.6960,  -9.2149,  -5.9667,   4.8722,
             1.9148,   5.2397,   6.2996],
          [ -4.6237,   5.0732,   0.0882,  -0.3067,  -9.4001,  -3.9526,   4.1384,
             3.7886,   6.0533,   5.6846],
          [ -2.3

In [38]:
rf=torch.randint(0,10,(3,6),dtype=torch.float32)
g.ndata['i']=rf


In [39]:
i=dgl.mean_nodes(g,'i')
print(i)

tensor([[4.3333, 6.6667, 3.0000, 3.6667, 1.6667, 4.0000]])
